<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/inference_lama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git

print('\n> Install dependencies')
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet

print('\n> Changing the dir to:')
% cd /content/lama

print('\n> Download the model')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/EgqaSnLohjuzAg) -o lama-models.zip
!unzip lama-models.zip

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet


print('\n> Init mask-drawing code')
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil

print('Downloading test data')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/xKQJZeVRk5vLlQ) -o LaMa_test_images.zip
!unzip LaMa_test_images.zip


> Cloning the repo
Cloning into 'lama'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 283 (delta 73), reused 264 (delta 66), pack-reused 0
Receiving objects: 100% (283/283), 6.49 MiB | 29.16 MiB/s, done.
Resolving deltas: 100% (73/73), done.

> Install dependencies
     |████████████████████████████████| 12.5 MB 2.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 72 kB 687 kB/s 
     |████████████████████████████████| 144 kB 42.9 MB/s 
     |████████████████████████████████| 841 kB 78.3 MB/s 
     |████████████████████████████████| 271 kB 89.8 MB/s 
     |████████████████████████████████| 47 kB 6.3 MB/s 
     |████████████████████████████████| 948 kB 77.8 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 112 kB 86.7 MB/s 
     |████████████████████████████████| 74 kB 4.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
# MODEL_PATH = '/content/gdrive/MyDrive/repos/GANime/data_out/lama_model/matt_2022-02-14_00-21-53_train_lama-fourier_/models/last.ckpt'
MODEL_PATH = '/content/lama/LaMa_models/lama-places/lama-fourier'
INPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/mask_sample.zip'
INPUT_DIR = '/content/input/'
INPUT_DIR_2 = '/content/input/mask_sample/'
OUTPUT_DIR = '/contetn/output/'
OUTPUT_ZIP = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/lama_output_sample.zip'
# $(pwd)/LaMa_models/lama-places/lama-fourier
# indir=$(pwd)/sample outdir=$(pwd)/output

shutil.unpack_archive(INPUT_ZIP, INPUT_DIR)

In [12]:
# run to do inference on samples from LaMa
print('Predicting')
# os.system(f'export TORCH_HOME=$(pwd) && export PYTHONPATH=. && python3 $(pwd)bin/predict.py model.path={MODEL_PATH} indir={INPUT_DIR_2} outdir={OUTPUT_DIR}')
!export TORCH_HOME=$(pwd) && export PYTHONPATH=. && python3 bin/predict.py model.path=$(pwd)/LaMa_models/lama-places/lama-fourier indir=$(pwd)/sample outdir=$(pwd)/output

Predicting
Detectron v2 is not installed
[2022-02-17 16:05:07,988][saicinpainting.utils][WARNING] - Setting signal 10 handler <function print_traceback_handler at 0x7f4865616440>
[2022-02-17 16:05:08,017][root][INFO] - Make training model default
[2022-02-17 16:05:08,018][saicinpainting.training.trainers.base][INFO] - BaseInpaintingTrainingModule init called
[2022-02-17 16:05:08,018][root][INFO] - Make generator ffc_resnet
[2022-02-17 16:05:08,276][saicinpainting.training.trainers.base][INFO] - Generator
FFCResNetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): FFC_BN_ACT(
      (ffc): FFC(
        (convl2l): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1), bias=False, padding_mode=reflect)
        (convl2g): Identity()
        (convg2l): Identity()
        (convg2g): Identity()
        (gate): Identity()
      )
      (bn_l): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn_g): Identity()
      (act_l): ReLU(inp

In [ ]:
shutil.unpack_archive('/content/sample.zip', '/content/lama/sample/')